# Visualizing Model Output - Part 2

#### Overview
In this notebook, we will continue looking at ways to visualize ocean model output. As in previous lessons, we will use output from the ECCO Ocean State Estimate (Version 5).

#### Import Modules
First, import the modules required to access data from netCDF files and create plots:

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean.cm as cm
import moviepy.video.io.ImageSequenceClip

### Defining the data path
As in the previous lesson, we will use data stored on our external drive. Begin by defining the path to your data folder:

In [2]:
# Define a path to a data folder
data_folder = 'C:\\Users\\tandre\\Desktop\\Spring 2025\\cs185c\\data'

## Download Daily Data
Following the steps established in Lecture 3-1, start by downloading all of the daily sea surface temperature data:

```
version = 'Version4'
release = 'Release4'
subset = 'interp_monthly'
var_name = 'SIarea'
start_year = 2015
end_year = 2015
```

In previous notebooks, we've plotted our data in longitude-latitude coordinates. Let's take a first look at some of the sea ice data by making a quick plot:

In [3]:
# make a path to a sea ice file
seaice_file = os.path.join(data_folder,'ECCO','Version4','Release4',
                          'interp_monthly','SIarea','SIarea_2015_01.nc')

# read in the sea ice data at along with the
# latitude and longitude information 
ds = xr.open_dataset(seaice_file)
longitude = np.array(ds['longitude'][:])
longitude
latitude = np.array(ds['latitude'][:])
SIarea = np.array(ds['SIarea'][:])
ds.close()

# subset sea ice to the first time step
SIarea = SIarea[0,:,:]

FileNotFoundError: [Errno 2] No such file or directory: b'C:\\Users\\tandre\\Desktop\\Spring 2025\\cs185c\\data\\ECCO\\Version4\\Release4\\interp_monthly\\SIarea\\SIarea_2015_01.nc'

If we plot the sea ice field as we did in lessons 2-1 and 2-2, we see a map that looks roughly like the globe:

In [ ]:
# create a figure object
fig = plt.figure()

# plot the temperature
plt.pcolormesh(longitude, latitude, SIarea, vmin=0, vmax=1, cmap=cm.ice)
plt.colorbar(orientation = 'horizontal')

# format the axes
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Sea Ice Fraction')
plt.show()

Looking at the sea ice data in this view is a little silly because our map is focused on the equator while the sea ice is at the poles. One way we could modify this is by changing the *projection* of our map.

## Part 2: Plotting with Cartopy
When we are working with numerical ocean model data, our numerical models represent locations on the globe. In these situations, it is helpful to plot our data in a projection that represents this aspect of our data. With this in mind, we'll invstigate plotting with the `cartopy` package which refers to **carto**graphy with **py**thon. 

In looking at the plot above, there's at least two things that are dissatisfying. First, the world is very distorted at the poles. Second, the continents are filled in with a default value of 0, which is a the same value given to areas of no sea ice - kinda confusing. We can remedy both of these using `cartopy` by choosing a better projection for our data and adding polygons that cover the coastline. 

Take a look at the plotting code below:

In [ ]:
# create a figure object
fig = plt.figure()
ax = plt.axes(projection=ccrs.Mollweide())

# plot the seaice
plt.pcolormesh(longitude, latitude, SIarea, vmin=0, vmax=1, cmap=cm.ice,
               transform=ccrs.PlateCarree())
plt.colorbar(orientation = 'horizontal')

# add coastlines
plt.gca().add_feature(cfeature.LAND, zorder=99, facecolor='silver')
plt.gca().coastlines()

# format the axes
plt.title('Sea Ice Fraction')
plt.show()

### &#x1F914; Spot the differences
What are the key differences between the code that generates the plot above compared to the previous plot?

A projection parameter is used to project the results of the plot onto a specified shape.
Also, plt.gca() is used to add features like coastlines and landmasses on the map.

### Projections
As you can see above, the axes object provides the projection system for the map. We see that a `projection` parameter has been set to a specific projection - in this case the `Robinson` projection. The `cartopy` package has a variety of different projections for plotting mapped data. Test some of the following common projections by modifying the plot above:

| Projection Code | Default Parameters |
|-----------------|--------------------|
| PlateCarree()  | central_longitude=0.0 |
| Mollweide()     | central_longitude=0.0 |
| Orthographic()  | central_longitude=0.0, central_latitude=0.0 |
| Robinson()      | central_longitude=0.0 |
| InterruptedGoodeHomolosine() | central_longitude=0.0 |

When you find your favorite projection, try changing the default central longitude/latitude to see how the plot changes.

### &#x2757; Note
In the plot above, the `transform=ccrs.PlateCarree()` keyword is crucial to plotting the data correctly. This line tells `cartopy` that the data is in longitude-latitude coordinates. Without this keyword, there is no way that `cartopy` will know how to put the data on the map.

## Part 2: Figures and Functions
In the subsequent sections, we're going to take a look at how to create movies in Python but first, we're going to build up some machinery to create the frames of our movie. After we've created our frames, it will be easy to string them together into a movie.

First, let's define a function called `plot_frame` to make a plot. This function should take in two input arguments: the sea ice field and a file path.

In [ ]:
# adapt the code below in to a function
# called plot_frame that takes SIarea and the file_path as arguments

def plot_frame(SIarea, file_path):

    #file_path=file_path
    # make a figure object with projection
    fig = plt.figure(figsize=(6,6))
    ax = plt.axes(projection=ccrs.Orthographic(central_latitude = 80))

    # plot the sea ice field
    plt.pcolormesh(longitude, latitude, SIarea, vmin=0, vmax=1, cmap=cm.ice,
               transform=ccrs.PlateCarree())
    plt.colorbar(orientation = 'horizontal', label='Sea Ice Concentration')

    # add coastlines
    plt.gca().add_feature(cfeature.LAND, zorder=99, facecolor='silver')
    plt.gca().coastlines()

    # format the axes
    plt.title('Sea Ice Fraction')

    # save the figure
    plt.savefig(file_path)
    plt.close(fig)

Once you're happy with your code, we can test it out with the data we've read in for Jan 15 above:

In [ ]:
# define a path to a directory for your frames directory
frames_directory = '.\\frames'

# define an output path for your frame
file_path = os.path.join(frames_directory,'SIarea_2015_01.png')

# test your plotting function here
plot_frame(SIarea, file_path)

In [ ]:
file_list = []
for i in range(1,13):
    name = 'SIarea_2015_' + '{:02d}'.format(i)
    
    # make a path to a sea ice file
    seaice_file = os.path.join(data_folder,'ECCO','Version4','Release4',
                              'interp_monthly','SIarea', name +'.nc')
    
    # read in the sea ice data at along with the
    # latitude and longitude information 
    ds = xr.open_dataset(seaice_file)
    longitude = np.array(ds['longitude'][:])
    latitude = np.array(ds['latitude'][:])
    SIarea = np.array(ds['SIarea'][:])
    ds.close()

    # subset sea ice to the first time step
    SIarea = SIarea[0,:,:]

    file_list.append(file_path)
    file_path=os.path.join(frames_directory, name + '.png')
    
    plot_frame(SIarea, file_path)

Now that we've created out images, we're ready to stitch them together to make a movie! We'll tackle this task in class on Tuesday.

In [ ]:
fps = 5

clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(file_list, fps = fps)
clip.write_videofile('Arctic Sea Ice.mp4')